In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
os.chdir("../../../../")

import yaml
import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon as mu
import numpy as np
import scanpy as sc
import scanpy.external as sce
from scipy import sparse
from matplotlib.colors import LinearSegmentedColormap

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=100,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)
import matplotlib as mpl

sc.set_figure_params(dpi=100)
sns.set(style="white")

In [ ]:
mudata = md.read_h5mu("data/anca/exploratory/ANCA_27PK27PB_cd4emrm_totalvi.h5mu")
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]
mudata

In [ ]:
cluster_key = "leiden_0.8"
get_cluster_names(mod_rna, cluster_key)

In [ ]:
sc.pl.umap(mod_rna, color=[cluster_key, "cell_type"], legend_loc="on data", wspace=0.6)

In [ ]:
plot_qc(mod_rna, "leiden_0.8")

In [ ]:
annotations = {
    "0": "Th17",
    "1": "Th1",
    "2": "Tfh",
    "3": "Th1-like",
    "4": "Tc1-like",
    "5": "Th17",
    "6": "Tfh",
    "7": "Th1-like",
    "8": "Th1",
    "9": "Th1",
    "10": "Th1-like",
    "11": "Th1-like",
}

In [ ]:
mod_rna.obs["cell_type_fine"] = mod_rna.obs[cluster_key].replace(annotations)
sc.pl.umap(mod_rna, color="cell_type_fine")

In [ ]:
mudata = md.MuData(
    {
        "rna": mod_rna,
        "cite": mod_cite,
    }
)
mudata.write_h5mu("data/anca/exploratory/ANCA_27PK27PB_cd4emrm_annotated.h5mu")

In [ ]:
cmap_cd4 = {
    "Th17": "#7B1717",
    "Th1": "#A91F1F",
    "Tfh": "#E0B093",
    "Th1-like": "#D62728",
    "Tc1-like": "#D2D2D2",
}

labels_cd4 = [
    "Th17",
    "Th1",
    "Th1-like",
    "Tfh",
    "Tc1-like",
]
plot_single_stacked_bar(
    mod_rna,
    labels=labels_cd4,
    color_map=cmap_cd4,
    celltype_col="cell_type_fine",
    save_path=None,
    ylabel="CD4+ Teff subset proportions",
)

In [ ]:
all_marker_results, filtered_marker_results, best_markers = run_de_pipeline(
    mod_rna,
    "cell_type_fine",
    mod="rna",
    top_n=5,
    min_expression=0.05,
)

## Protein markers

In [ ]:
mod_cite.obs["cell_type_fine"] = mod_rna.obs["cell_type_fine"]
mod_cite.obsm["X_umap"] = mod_rna.obsm["X_umap"]
protein_clr(mod_cite)

In [ ]:
all_marker_results, filtered_marker_results, best_markers = run_de_pipeline(
    mod_cite,
    "cell_type_fine",
    mod="protein",
    top_n=5,
    min_expression=0.1,
)